In [1]:
#Import the libraries
import numpy as np
import pandas as pd

import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_data = pickle.load(open('clean_dataset_2022/train_set.bin', 'rb'))

test_data = pickle.load(open('clean_dataset_2022/test_set.bin', 'rb'))

## SARIMAX

In [3]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

from matplotlib.pyplot import figure

### Resample

In [4]:
province = ['BKK','CNX','KKC','RAY','SARA','SURAT']

In [5]:
for p in province:
    train_data[p] = train_data[p].resample('6H').mean()

    test_data[p] = test_data[p].resample('6H').mean()

### Split 70% 30%

In [6]:
train_set = {} ; valid_set = {}

ratio = 0.7

for p in province:
    train_size, valid_size = int(ratio*train_data[p].shape[0]), int((1-ratio)*train_data[p].shape[0])
    train_set[p], valid_set[p] = train_data[p].iloc[:train_size], train_data[p].iloc[train_size: ]

### Tuning Parameters

In [7]:
order = (1, 0, 1)
seasonal_order = (0, 1, 0, 1461) # 365.25 * 4

exog_order = (5, 0, 4)
exog_seasonal_order = (1, 0, 1, 1461) # 365.25 * 4

exog_columns = ['Temp', 'WindSpeed', 'WindDir']

### Training 6 provinces with *minimal_SARIMAX*

In [8]:
from importlib import reload

In [9]:
from custom_function import minimalSARIMAX

reload(minimalSARIMAX)

from custom_function.minimalSARIMAX import MinimalSARIMAX

In [10]:
model = {}
model_exog = {}

for p in province:
    model[p] = MinimalSARIMAX(train_data[p][['PM25']],
                order,
                seasonal_order,
                exog=train_data[p][exog_columns])
    
    model_exog[p] = {}    
    for exog in exog_columns:
        model_exog[p][exog] = MinimalSARIMAX(train_data[p][[exog]],
                        exog_order,
                        exog_seasonal_order)

In [11]:
for p in province:
    model[p].fit(lr=1e-5, lr_decay=0.999 ,verbose=0)

100%|██████████| 2555/2555 [00:00<00:00, 4094.50it/s]


In [12]:
for p in province:
    for exog in exog_columns:
        model_exog[p][exog].fit(lr=1e-5, lr_decay=0.999 ,verbose=0)

100%|██████████| 2555/2555 [00:00<00:00, 3797.93it/s]


In [13]:
y_pred = {} ; Err = {}

In [14]:
prev_data = {}
prev_exog = {}

y_pred['BKK'], Err['BKK'] = model['BKK'].predict(test_data['BKK'][['PM25']], y_X=test_data['BKK'][exog_columns], verbose=0)

In [15]:
model['BKK'].RMSE(y_pred['BKK'], test_data['BKK'][['PM25']])

Test on SARIMAX with RMSE: 3.902526916621347


In [22]:
df_test = valid_set['BKK'].copy()

In [ ]:
y_t = df_test.iloc[:,[0]].to_numpy().ravel()

y_Xt = df_test[exog_columns].to_numpy()

y_pred = [10]

Error = [y_t[0]-10]


verbose = 0

for t in range(1,len(y_t)):
    pred = {} ; x = {}
    pred['p'], x['p'] = model['BKK'].p_prediction(y_t, t)
    pred['q'], x['q'] = model['BKK'].q_prediction(y_t, Error, t)
    pred['pX'], x['pX'] = model['BKK'].pX_prediction(y_Xt, t)
    pred['P'], x['P'] = model['BKK'].P_prediction(y_t, t)
    pred['Q'], x['Q'] = model['BKK'].Q_prediction(y_t, Error, t)

    pred['y'] = (pred['p'] + pred['q'] + pred['P'] + pred['Q'] + model['BKK'].params['c']).sum()
    
    y_pred.append(pred['y'])

    error_t = y_t[t] - pred['y']

    if verbose:
        print(t, pred['y'], y_t[t], error_t)

    Error.append(error_t)

y_pred_tmp = df_test.iloc[:,[0]].copy()
y_pred_tmp['PM25'] = np.array(y_pred)

# Grid Search

In [16]:
import itertools

In [17]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 2)]

In [18]:
pdqs = [(x[0], x[1], x[2], 1461) for x in list(itertools.product(p, d, q))]
pdqs

[(0, 0, 0, 1461),
 (0, 0, 1, 1461),
 (0, 0, 2, 1461),
 (0, 1, 0, 1461),
 (0, 1, 1, 1461),
 (0, 1, 2, 1461),
 (0, 2, 0, 1461),
 (0, 2, 1, 1461),
 (0, 2, 2, 1461),
 (1, 0, 0, 1461),
 (1, 0, 1, 1461),
 (1, 0, 2, 1461),
 (1, 1, 0, 1461),
 (1, 1, 1, 1461),
 (1, 1, 2, 1461),
 (1, 2, 0, 1461),
 (1, 2, 1, 1461),
 (1, 2, 2, 1461),
 (2, 0, 0, 1461),
 (2, 0, 1, 1461),
 (2, 0, 2, 1461),
 (2, 1, 0, 1461),
 (2, 1, 1, 1461),
 (2, 1, 2, 1461),
 (2, 2, 0, 1461),
 (2, 2, 1, 1461),
 (2, 2, 2, 1461)]

In [24]:
# Define function
def sarimax_gridsearch(y_train, y_test, pdq, PDQs, x_train = None, x_test = None):
    '''
    Input: 
        pm_train: PM2.5 training data
        exo_train: exogenous training data
        pdq : ARIMA combinations 
        pdqs : seasonal ARIMA combinations 

    Return:
        Prints out top 5 parameter combinations
        Returns dataframe of parameter combinations ranked by RMSE
    '''
    ans = []
    for comb in pdq:
        for combs in PDQs:
            p, d, q = comb[0], comb[1], comb[2]
            P, D, Q = combs[0], combs[1], combs[2]
            if (p+q <= 2) and (d <= 2) and (d+D <= 2) and (P+Q <= 1):  
                try:
                    model = MinimalSARIMAX(y_train, comb, combs, exog=x_train)
                    model.fit(lr=1e-5, lr_decay=0.999 ,verbose=0) 
                    y_pred, err = model.predict(y_test, y_X=x_test, verbose=0)
                    y_pred = pd.DataFrame(y_pred['PM25'])
                    rmse = model.scoring(y_pred, y_test)
                    ans.append([comb, combs, rmse])
                    # print(f'SARIMAX {comb} x {combs} : RMSE Calculated ={rmse}')
                except Exception as e: 
                    # print(e)
                    continue

    # Convert into a dataframe
    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'rmse'])

    # Sort and return top 5 combinations
    ans_df = ans_df.sort_values(by=['rmse'],ascending=True)
    
    return ans_df.iloc[0]

In [20]:
pm_train_bkk = train_data[province[0]][['PM25']]
pm_test_bkk = test_data[province[0]][['PM25']]
exo_train_bkk = train_data[province[0]][exog_columns]
exo_test_bkk = test_data[province[0]][exog_columns]

pm_train_cnx = train_data[province[1]][['PM25']]
pm_test_cnx = test_data[province[1]][['PM25']]
exo_train_cnx = train_data[province[1]][exog_columns]
exo_test_cnx = test_data[province[1]][exog_columns]

pm_train_kkc = train_data[province[2]][['PM25']]
pm_test_kkc = test_data[province[2]][['PM25']]
exo_train_kkc = train_data[province[2]][exog_columns]
exo_test_kkc = test_data[province[2]][exog_columns]

pm_train_ray = train_data[province[3]][['PM25']]
pm_test_ray = test_data[province[3]][['PM25']]
exo_train_ray = train_data[province[3]][exog_columns]
exo_test_ray = test_data[province[3]][exog_columns]

pm_train_sara = train_data[province[4]][['PM25']]
pm_test_sara = test_data[province[4]][['PM25']]
exo_train_sara = train_data[province[4]][exog_columns]
exo_test_sara = test_data[province[4]][exog_columns]

pm_train_surat = train_data[province[5]][['PM25']]
pm_test_surat = test_data[province[5]][['PM25']]
exo_train_surat = train_data[province[5]][exog_columns]
exo_test_surat = test_data[province[5]][exog_columns]

## Tuning parameters for PM2.5

In [ ]:
result_bkk = sarimax_gridsearch(pm_train_bkk, pm_test_bkk, pdq, pdqs, exo_train_bkk, exo_test_bkk)
result_cnx = sarimax_gridsearch(pm_train_cnx, pm_test_cnx, pdq, pdqs, exo_train_cnx, exo_test_cnx)
result_kkc = sarimax_gridsearch(pm_train_kkc, pm_test_kkc, pdq, pdqs, exo_train_kkc, exo_test_kkc)
result_ray = sarimax_gridsearch(pm_train_ray, pm_test_ray, pdq, pdqs, exo_train_ray, exo_test_ray)
result_sara = sarimax_gridsearch(pm_train_sara, pm_test_sara, pdq, pdqs, exo_train_sara, exo_test_sara)
result_surat = sarimax_gridsearch(pm_train_surat, pm_test_surat, pdq, pdqs, exo_train_surat, exo_test_surat)

In [29]:
order = {province[0]: result_bkk.pdq, province[1]: result_cnx.pdq, province[2]: result_kkc.pdq, province[3]: result_ray.pdq, province[4]: result_sara.pdq, province[5]: result_surat.pdq}
seasonal_order = {province[0]: result_bkk.pdqs, province[1]: result_cnx.pdqs, province[2]: result_kkc.pdqs, province[3]: result_ray.pdqs, province[4]: result_sara.pdqs, province[5]: result_surat.pdqs}
print(order)
print(seasonal_order)

{'BKK': (2, 1, 0), 'CNX': (1, 0, 1), 'KKC': (2, 1, 0), 'RAY': (2, 1, 0), 'SARA': (2, 0, 0), 'SURAT': (2, 1, 0)}
{'BKK': (1, 1, 0, 1461), 'CNX': (0, 1, 1, 1461), 'KKC': (0, 1, 0, 1461), 'RAY': (0, 1, 0, 1461), 'SARA': (0, 0, 0, 1461), 'SURAT': (0, 1, 1, 1461)}


## Tuning parameters for temperature

In [33]:
temp = exog_columns[0]
temp_train_bkk = pd.DataFrame(exo_train_bkk[temp])
temp_test_bkk = pd.DataFrame(exo_test_bkk[temp])

temp_train_cnx = pd.DataFrame(exo_train_cnx[temp])
temp_test_cnx = pd.DataFrame(exo_test_cnx[temp])

temp_train_kkc = pd.DataFrame(exo_train_kkc[temp])
temp_test_kkc = pd.DataFrame(exo_test_kkc[temp])

temp_train_ray = pd.DataFrame(exo_train_ray[temp])
temp_test_ray = pd.DataFrame(exo_test_ray[temp])

temp_train_sara = pd.DataFrame(exo_train_sara[temp])
temp_test_sara = pd.DataFrame(exo_test_sara[temp])

temp_train_surat = pd.DataFrame(exo_train_surat[temp])
temp_test_surat = pd.DataFrame(exo_test_surat[temp])

In [ ]:
gSearch_temp_bkk = sarimax_gridsearch(temp_train_bkk, temp_test_bkk, pdq, pdqs)
gSearch_temp_cnx = sarimax_gridsearch(temp_train_cnx, temp_test_cnx, pdq, pdqs)
gSearch_temp_kkc = sarimax_gridsearch(temp_train_kkc, temp_test_kkc, pdq, pdqs)
gSearch_temp_ray = sarimax_gridsearch(temp_train_ray, temp_test_ray, pdq, pdqs)
gSearch_temp_sara = sarimax_gridsearch(temp_train_sara, temp_test_sara, pdq, pdqs)
gSearch_temp_surat = sarimax_gridsearch(temp_train_surat, temp_test_surat, pdq, pdqs)

In [36]:
temp_order = {province[0]: gSearch_temp_bkk.pdq, province[1]: gSearch_temp_cnx.pdq, province[2]: gSearch_temp_kkc.pdq, province[3]: gSearch_temp_ray.pdq, province[4]: gSearch_temp_sara.pdq, province[5]: gSearch_temp_surat.pdq}
temp_seasonal_order = {province[0]: gSearch_temp_bkk.pdqs, province[1]: gSearch_temp_cnx.pdqs, province[2]: gSearch_temp_kkc.pdqs, province[3]: gSearch_temp_ray.pdqs, province[4]: gSearch_temp_sara.pdqs, province[5]: gSearch_temp_surat.pdqs}
print(temp_order)
print(temp_seasonal_order)

{'BKK': (1, 0, 0), 'CNX': (1, 1, 0), 'KKC': (1, 0, 0), 'RAY': (1, 0, 0), 'SARA': (1, 0, 0), 'SURAT': (1, 0, 0)}
{'BKK': (0, 0, 0, 1461), 'CNX': (0, 0, 0, 1461), 'KKC': (0, 0, 0, 1461), 'RAY': (0, 0, 1, 1461), 'SARA': (0, 0, 0, 1461), 'SURAT': (0, 0, 1, 1461)}


## Tuning parameters for windspeed

In [38]:
windSpeed = exog_columns[1]
windSpeed_train_bkk = pd.DataFrame(exo_train_bkk[windSpeed])
windSpeed_test_bkk = pd.DataFrame(exo_test_bkk[windSpeed])

windSpeed_train_cnx = pd.DataFrame(exo_train_cnx[windSpeed])
windSpeed_test_cnx = pd.DataFrame(exo_test_cnx[windSpeed])

windSpeed_train_kkc = pd.DataFrame(exo_train_kkc[windSpeed])
windSpeed_test_kkc = pd.DataFrame(exo_test_kkc[windSpeed])

windSpeed_train_ray = pd.DataFrame(exo_train_ray[windSpeed])
windSpeed_test_ray = pd.DataFrame(exo_test_ray[windSpeed])

windSpeed_train_sara = pd.DataFrame(exo_train_sara[windSpeed])
windSpeed_test_sara = pd.DataFrame(exo_test_sara[windSpeed])

windSpeed_train_surat = pd.DataFrame(exo_train_surat[windSpeed])
windSpeed_test_surat = pd.DataFrame(exo_test_surat[windSpeed])

In [39]:
gSearch_windSpeed_bkk = sarimax_gridsearch(windSpeed_train_bkk, windSpeed_test_bkk, pdq, pdqs)
gSearch_windSpeed_cnx = sarimax_gridsearch(windSpeed_train_cnx, windSpeed_test_cnx, pdq, pdqs)
gSearch_windSpeed_kkc = sarimax_gridsearch(windSpeed_train_kkc, windSpeed_test_kkc, pdq, pdqs)
gSearch_windSpeed_ray = sarimax_gridsearch(windSpeed_train_ray, windSpeed_test_ray, pdq, pdqs)
gSearch_windSpeed_sara = sarimax_gridsearch(windSpeed_train_sara, windSpeed_test_sara, pdq, pdqs)
gSearch_windSpeed_surat = sarimax_gridsearch(windSpeed_train_surat, windSpeed_test_surat, pdq, pdqs)

  0%|          | 0/2555 [00:00<?, ?it/s]


In [40]:
windSpeed_order = {province[0]: gSearch_windSpeed_bkk.pdq, province[1]: gSearch_windSpeed_cnx.pdq, province[2]: gSearch_windSpeed_kkc.pdq, province[3]: gSearch_windSpeed_ray.pdq, province[4]: gSearch_windSpeed_sara.pdq, province[5]: gSearch_windSpeed_surat.pdq}
windSpeed_seasonal_order = {province[0]: gSearch_windSpeed_bkk.pdqs, province[1]: gSearch_windSpeed_cnx.pdqs, province[2]: gSearch_windSpeed_kkc.pdqs, province[3]: gSearch_windSpeed_ray.pdqs, province[4]: gSearch_windSpeed_sara.pdqs, province[5]: gSearch_windSpeed_surat.pdqs}
print(windSpeed_order)
print(windSpeed_seasonal_order)

{'BKK': (1, 1, 0), 'CNX': (1, 1, 0), 'KKC': (1, 1, 0), 'RAY': (1, 1, 0), 'SARA': (1, 0, 0), 'SURAT': (1, 1, 0)}
{'BKK': (0, 1, 1, 1461), 'CNX': (0, 1, 1, 1461), 'KKC': (0, 1, 1, 1461), 'RAY': (0, 1, 1, 1461), 'SARA': (0, 0, 0, 1461), 'SURAT': (0, 1, 1, 1461)}


## Tuning parameters for wind direction

In [44]:
windDir = exog_columns[2]
windDir_train_bkk = pd.DataFrame(exo_train_bkk[windDir])
windDir_test_bkk = pd.DataFrame(exo_test_bkk[windDir])

windDir_train_cnx = pd.DataFrame(exo_train_cnx[windDir])
windDir_test_cnx = pd.DataFrame(exo_test_cnx[windDir])

windDir_train_kkc = pd.DataFrame(exo_train_kkc[windDir])
windDir_test_kkc = pd.DataFrame(exo_test_kkc[windDir])

windDir_train_ray = pd.DataFrame(exo_train_ray[windDir])
windDir_test_ray = pd.DataFrame(exo_test_ray[windDir])

windDir_train_sara = pd.DataFrame(exo_train_sara[windDir])
windDir_test_sara = pd.DataFrame(exo_test_sara[windDir])

windDir_train_surat = pd.DataFrame(exo_train_surat[windDir])
windDir_test_surat = pd.DataFrame(exo_test_surat[windDir])

In [45]:
gSearch_windDir_bkk = sarimax_gridsearch(windDir_train_bkk, windDir_test_bkk, pdq, pdqs)
gSearch_windDir_cnx = sarimax_gridsearch(windDir_train_cnx, windDir_test_cnx, pdq, pdqs)
gSearch_windDir_kkc = sarimax_gridsearch(windDir_train_kkc, windDir_test_kkc, pdq, pdqs)
gSearch_windDir_ray = sarimax_gridsearch(windDir_train_ray, windDir_test_ray, pdq, pdqs)
gSearch_windDir_sara = sarimax_gridsearch(windDir_train_sara, windDir_test_sara, pdq, pdqs)
gSearch_windDir_surat = sarimax_gridsearch(windDir_train_surat, windDir_test_surat, pdq, pdqs)

  0%|          | 0/2555 [00:00<?, ?it/s]


In [46]:
windDir_order = {province[0]: gSearch_windDir_bkk.pdq, province[1]: gSearch_windDir_cnx.pdq, province[2]: gSearch_windDir_kkc.pdq, province[3]: gSearch_windDir_ray.pdq, province[4]: gSearch_windDir_sara.pdq, province[5]: gSearch_windDir_surat.pdq}
windDir_seasonal_order = {province[0]: gSearch_windDir_bkk.pdqs, province[1]: gSearch_windDir_cnx.pdqs, province[2]: gSearch_windDir_kkc.pdqs, province[3]: gSearch_windDir_ray.pdqs, province[4]: gSearch_windDir_sara.pdqs, province[5]: gSearch_windDir_surat.pdqs}
print(windDir_order)
print(windDir_seasonal_order)

{'BKK': (1, 1, 0), 'CNX': (1, 1, 0), 'KKC': (2, 1, 0), 'RAY': (1, 0, 1), 'SARA': (1, 1, 0), 'SURAT': (1, 1, 1)}
{'BKK': (0, 1, 1, 1461), 'CNX': (1, 1, 0, 1461), 'KKC': (1, 1, 0, 1461), 'RAY': (0, 1, 0, 1461), 'SARA': (0, 0, 1, 1461), 'SURAT': (0, 1, 0, 1461)}
